In [1]:
from io import StringIO
from scipy.io import mmread
from scipy.io import mminfo
from google.colab import drive
drive.mount("/content/drive")
datAddr = "/content/drive/MyDrive/CMSC818J_Test/"
from scipy.sparse import csr_matrix
from scipy import sparse
import numpy as np
import sys
import os
import csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Baseline Architecture 
 

### Assumptions

1.   Each multiplication and accumulation takes 4 cycles
2.   Reading the rows from memory takes 4 cycles
3.   Storing the output takes 2 cycle 
4. PEs can read, compute and write back in parallel





In [2]:
def add_idle_cycles (cycles_num):
  for i in range(total_PEs):
    PE_stats[i][IDLE_CYCLES] += cycles_num 

In [3]:
def add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE):
  for i in range(total_PEs):
    PE_stats[i][IDLE_CYCLES] += max_nonzero_per_PE - idle_cycle_list[i] 

In [4]:
# Simulates PE MAC operations
def PE_mult(output_col_num):
  PE_to_process = PE_lst.pop(0)
  row_num = PE_output_index.pop(0) 
  accumulation = 0
  # optimization for inner product: whichever dictionary has smallest non-zeros, we iterate over that. 
  if len(B_dict) < len(PE_to_process):
    for col,data in B_dict.items():
      if col in PE_to_process:
        # multiply and then accumulate  
        accumulation += PE_to_process[col] * data 
        PE_stats[PE_num][BUSY_CYCLES] += 4
    output_mat2[row_num][output_col_num] = accumulation 
  else:
    for col,data in PE_to_process.items():
      if col in B_dict:
        # multiply and then accumulate  
        accumulation += B_dict[col] * data 
        PE_stats[PE_num][BUSY_CYCLES] += 4 

    output_mat2[row_num][output_col_num] = accumulation
    # Update PE stats
    
    
  if len(PE_to_process) > PE_stats[PE_num][MAX_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MAX_NUM_NON_ZEROS] = len(PE_to_process)
  elif len(PE_to_process) < PE_stats[PE_num][MIN_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MIN_NUM_NON_ZEROS] = len(PE_to_process)




In [5]:

## ENUMS for PE_stats
BUSY_CYCLES = 1
MAX_NUM_NON_ZEROS = 2
MIN_NUM_NON_ZEROS = 3
IDLE_CYCLES = 4
CYCLE_TO_READ = 4
CYCLES_TO_ADD = 4 # per element 
CYCLES_TO_ACCUMULATE = 2 

In [6]:
def sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index, TOTAL_NUMBER_OF_CYCLES):
  PE_rows_read = dict() 
  PE_rows_index = dict()
  max_nonzero_per_PE = 0 # keeps track of max number of nonzero per PE in one allocation cycle
  idle_cycle_list = [] # keeps track of idle cycles for each PE per allocation

  for i in range(total_PEs):
    idle_cycle_list.append(0)

  for i in range(total_PEs):
    if pointer_index_A < mat_csrA_len:
      numOfElems = (mat_csr.indptr[pointer_index_A + 1] - mat_csr.indptr[pointer_index_A])
      data_row_A = mat_csr.data[data_index_A:numOfElems+data_index_A]
      row_A = mat_csr.indices[col_index_A:numOfElems+data_index_A]
      PE_lst.append(dict(zip(row_A, data_row_A)))
      PE_output_index.append(output_row_index)
      data_index_A += numOfElems
      col_index_A += numOfElems
      pointer_index_A += 1
      output_row_index += 1
      
      # find the max number of non-zeroes in all the PEs. Encodes that 
      # the PEs have to wait for each other. 
      if output_row_index % total_PEs != 0 :
        max_nonzero_per_PE = max(max_nonzero_per_PE, numOfElems)
        idle_cycle_list[output_row_index % total_PEs] = numOfElems 
      else:
        TOTAL_NUMBER_OF_CYCLES += max_nonzero_per_PE * CYCLES_TO_ADD
        TOTAL_NUMBER_OF_CYCLES += CYCLES_TO_ACCUMULATE # For accumulation 
        add_idle_cycles2(idle_cycle_list, max_nonzero_per_PE)
        max_nonzero_per_PE = 0  
       
  return (data_index_A, col_index_A, pointer_index_A, output_row_index, TOTAL_NUMBER_OF_CYCLES)

In [ ]:
directory = os.fsencode(datAddr)

for pe_num in range(2, 10, 2):

  with open('/content/drive/MyDrive/CMSC818J_Data/Baseline_PE' + str(pe_num) + '.csv', 'w', newline='') as file:
    writer = csv.writer(file)    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)

        ## Read the file 
        m = mmread(datAddr+filename)
        mat_csr = m.tocsr()
        mat_csrB = m.tocsc()

        ## Init for PE stats 
        total_PEs = pe_num # number of PEs available. Can change this number as required
        PE_stats = []
        for i in range(total_PEs):
          PE_stat = dict()
          PE_stat[BUSY_CYCLES] = 0  
          PE_stat[MAX_NUM_NON_ZEROS] = 0 
          PE_stat[MIN_NUM_NON_ZEROS ] = sys.maxsize
          PE_stat[IDLE_CYCLES] = 0
          PE_stats.append(PE_stat)
        PE_stats


        mat_csrB_len = len(mat_csrB.indptr) - 1
        mat_csrA_len = len(mat_csr.indptr) - 1
        output_mat_len = len(mat_csrB.indptr) - 1
        PE_num = 0
        TOTAL_NUMBER_OF_CYCLES = 0

        TOTAL_NUMBER_OF_CYCLES += CYCLE_TO_READ # For reading elements, one time cost rest of them are amortized with PE computations
        add_idle_cycles(CYCLE_TO_READ)
        # acts as the buffer that stores rows to be allocated to PEs
        PE_lst = []
        PE_output_index = []
        # matrix to store the file output 
        output_mat2 = [[0 for _ in range(output_mat_len)] for _ in range(output_mat_len)]
        data_index_B = 0 
        row_index_B = 0
        pointer_index_B = 0 
        output_col_index = 0
        output_row_index = 0


        for j in range(mat_csrB_len): 
          # Decode col of matrix B from csc format
          numOfElems_B = (mat_csrB.indptr[pointer_index_B + 1] - mat_csrB.indptr[pointer_index_B])
          data_row_B = mat_csrB.data[data_index_B:numOfElems_B+data_index_B]
          col_B = mat_csrB.indices[row_index_B:numOfElems_B+data_index_B]

          # The decoded cols and corresponding data values are zipped into a dictionary 
          # data structure to easily pattern match with rows when computing inner product
          B_dict = dict(zip(col_B, data_row_B))
          data_index_B += numOfElems_B
          row_index_B += numOfElems_B
          pointer_index_B += 1
          data_index_A = 0
          col_index_A = 0
          pointer_index_A = 0
          output_row_index = 0
          for i in range(mat_csrA_len):
            # Decode rows of matrix A from csr format 
            if not PE_lst:
              # TOTAL_NUMBER_OF_CYCLES += 1 
              # add_idle_cycles(1, -1)
              data_index_A, col_index_A, pointer_index_A, output_row_index, TOTAL_NUMBER_OF_CYCLES = sort_computer_rows(data_index_A, col_index_A, pointer_index_A, output_row_index, TOTAL_NUMBER_OF_CYCLES)
            # allocate it to a PE using round-robin strategy 
            PE_mult(output_col_index)
            PE_num = (PE_num + 1) % total_PEs 
          output_col_index += 1 

        output_mat2
        output_mat2_reshape = np.reshape(output_mat2, (output_mat_len, output_mat_len))
        correct_output = np.dot(m.todense(),m.todense())
        # Validates if matrix multiplication is done correctly
        print(np.allclose(output_mat2_reshape, correct_output))
        # prints the number of cycles per PE, max and min number of zeros per PE
        string = [filename, TOTAL_NUMBER_OF_CYCLES]
        for pe in range(pe_num):
          for stat_num in range(1, 5, 1):
            string.append(PE_stats[pe][stat_num])

        writer.writerow(string)
      



